In [16]:
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig, BertForMaskedLM, BertTokenizer

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class BERTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.config = BertConfig(
            vocab_size=vocab_size,
            hidden_size=n_embd,
            num_hidden_layers=n_layer,
            num_attention_heads=n_head,
            intermediate_size=4 * n_embd,
            max_position_embeddings=block_size,
            dropout_rate=dropout
        )
        self.bert = BertModel(self.config)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        outputs = self.bert(idx)
        sequence_output = outputs[0]
        logits = self.lm_head(sequence_output)

        if targets is None:
            loss = None
        else:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, vocab_size), targets.view(-1))

        return logits, loss

model = BERTLanguageModel()
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForMaskedLM.from_pretrained("bert-base-uncased")

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Define the context for text generation
input_context = "The man walked [MASK] the street."
encoded_context = tokenizer(input_context, return_tensors="pt")

# Generate the masked word prediction
outputs = bert_model(**encoded_context)
predictions = outputs.logits.argmax(-1)
decoded_text = tokenizer.decode(predictions[0])

# Print the decoded text
print(decoded_text)

0.214785 M parameters


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


step 0: train loss 4.3491, val loss 4.3486
step 100: train loss 2.5911, val loss 2.6034
step 200: train loss 2.4126, val loss 2.4182
step 300: train loss 2.3115, val loss 2.3159
step 400: train loss 2.2287, val loss 2.2299
step 500: train loss 1.4462, val loss 1.4661
step 600: train loss 0.1396, val loss 0.1421
step 700: train loss 0.0970, val loss 0.0968
step 800: train loss 0.0912, val loss 0.0919
step 900: train loss 0.0877, val loss 0.0880
step 1000: train loss 0.0876, val loss 0.0883
step 1100: train loss 0.0843, val loss 0.0842
step 1200: train loss 0.0848, val loss 0.0851
step 1300: train loss 0.0846, val loss 0.0843
step 1400: train loss 0.0833, val loss 0.0832
step 1500: train loss 0.0827, val loss 0.0828
step 1600: train loss 0.0818, val loss 0.0818
step 1700: train loss 0.0818, val loss 0.0817
step 1800: train loss 0.0823, val loss 0.0833
step 1900: train loss 0.0820, val loss 0.0818
step 2000: train loss 0.0826, val loss 0.0812
step 2100: train loss 0.0799, val loss 0.0797


In [ ]:
# Test 1: Coherence Test
prompt_coherence = "Once upon a time, there was a brave knight named"
encoded_coherence = tokenizer(prompt_coherence, return_tensors="pt")
outputs_coherence = bert_model(**encoded_coherence)
predictions_coherence = outputs_coherence.logits.argmax(-1)
decoded_text_coherence = tokenizer.decode(predictions_coherence[0])
print("Coherence Test - Generated text:", decoded_text_coherence)

# Test 2: Contextual Relevance Test
prompt_relevance = "Romeo stood beneath Juliet's balcony and whispered,"
encoded_relevance = tokenizer(prompt_relevance, return_tensors="pt")
outputs_relevance = bert_model(**encoded_relevance)
predictions_relevance = outputs_relevance.logits.argmax(-1)
decoded_text_relevance = tokenizer.decode(predictions_relevance[0])
print("Contextual Relevance Test - Generated text:", decoded_text_relevance)

# Test 3: Stylistic Alignment Test
prompt_stylistic = "To be or not to be, that is"
encoded_stylistic = tokenizer(prompt_stylistic, return_tensors="pt")
outputs_stylistic = bert_model(**encoded_stylistic)
predictions_stylistic = outputs_stylistic.logits.argmax(-1)
decoded_text_stylistic = tokenizer.decode(predictions_stylistic[0])
print("Stylistic Alignment Test - Generated text:", decoded_text_stylistic)

# Test 4: Complexity Test
prompt_complexity = "In the darkest depths of the enchanted forest, the"
encoded_complexity = tokenizer(prompt_complexity, return_tensors="pt")
outputs_complexity = bert_model(**encoded_complexity)
predictions_complexity = outputs_complexity.logits.argmax(-1)
decoded_text_complexity = tokenizer.decode(predictions_complexity[0])
print("Complexity Test - Generated text:", decoded_text_complexity)

# Test 5: Cultural Reference Test
prompt_cultural = "Hark! What light through yonder window"
encoded_cultural = tokenizer(prompt_cultural, return_tensors="pt")
outputs_cultural = bert_model(**encoded_cultural)
predictions_cultural = outputs_cultural.logits.argmax(-1)
decoded_text_cultural = tokenizer.decode(predictions_cultural[0])
print("Cultural Reference Test - Generated text:", decoded_text_cultural)